# Agentes de Azure AI con Soporte para el Protocolo de Contexto del Modelo (MCP) - Python

Este cuaderno demuestra cómo usar los Agentes de Azure AI con herramientas del Protocolo de Contexto del Modelo (MCP) en Python. Muestra cómo crear un agente inteligente que puede aprovechar servidores MCP externos (como Microsoft Learn) para capacidades mejoradas utilizando autenticación sin claves.


## Instalar los Paquetes de Python Requeridos

Primero, necesitamos instalar los paquetes de Python necesarios:
- **azure-ai-projects**: SDK principal de Azure AI Projects
- **azure-ai-agents**: SDK de Azure AI Agents para crear y gestionar agentes
- **azure-identity**: Proporciona autenticación sin claves utilizando DefaultAzureCredential
- **mcp**: Implementación del Protocolo de Contexto de Modelos para Python


## Beneficios de la Autenticación Sin Claves

Este cuaderno demuestra la **autenticación sin claves**, que ofrece varias ventajas:
- ✅ **Sin claves de API que gestionar** - Utiliza autenticación basada en identidad de Azure
- ✅ **Mayor seguridad** - No se almacenan secretos en el código ni en archivos de configuración
- ✅ **Rotación automática de credenciales** - Azure gestiona el ciclo de vida de las credenciales
- ✅ **Control de acceso basado en roles** - Utiliza Azure RBAC para permisos detallados
- ✅ **Soporte para múltiples entornos** - Funciona sin problemas tanto en desarrollo como en producción

`DefaultAzureCredential` selecciona automáticamente la mejor fuente de credenciales disponible:
1. **Identidad Administrada** (cuando se ejecuta en Azure)
2. Credenciales de **Azure CLI** (durante el desarrollo local)
3. Credenciales de **Visual Studio**
4. **Variables de entorno** (si están configuradas)
5. Autenticación mediante **navegador interactivo** (como último recurso)


## Configuración de Autenticación Sin Claves

**Requisitos previos para la autenticación sin claves:**

### Para Desarrollo Local:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Para Entornos de Azure:
- Habilita la **Identidad Administrada Asignada por el Sistema** en tu recurso de Azure
- Asigna los **roles RBAC** apropiados a la identidad administrada:
  - `Cognitive Services OpenAI User` para acceso a Azure OpenAI
  - `AI Developer` para acceso a Proyectos de Azure AI

### Variables de Entorno (Opcional):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**¡No se necesitan claves API ni cadenas de conexión!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importar Bibliotecas Necesarias

Importa los módulos de Python necesarios:  
- **os, time**: Bibliotecas estándar de Python para variables de entorno y retrasos  
- **AIProjectClient**: Cliente principal para Proyectos de IA de Azure  
- **DefaultAzureCredential**: Autenticación sin claves para servicios de Azure  
- **Clases relacionadas con MCP**: Para crear y gestionar herramientas MCP y manejar aprobaciones  


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Configurar la configuración del servidor MCP

Configura la configuración del servidor MCP utilizando variables de entorno con valores predeterminados como respaldo:
- **MCP_SERVER_URL**: La URL del servidor MCP (por defecto, la API de Microsoft Learn)
- **MCP_SERVER_LABEL**: Una etiqueta para identificar el servidor MCP (por defecto, "mslearn")

Este enfoque permite una configuración flexible en diferentes entornos.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Crear Cliente de Proyecto de Azure AI (Autenticación sin Claves)

Inicializa el cliente del proyecto de Azure AI utilizando **autenticación sin claves**:
- **endpoint**: La URL del endpoint del proyecto de Azure AI Foundry
- **credential**: Usa `DefaultAzureCredential()` para una autenticación segura y sin claves
- **No se requieren claves de API**: Descubre y utiliza automáticamente la mejor credencial disponible

**Flujo de Autenticación:**
1. Verifica la Identidad Administrada (en entornos de Azure)
2. Recurre a las credenciales de Azure CLI (para desarrollo local)
3. Utiliza otras fuentes de credenciales disponibles según sea necesario

Este enfoque elimina la necesidad de gestionar claves de API o cadenas de conexión en tu código.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Crear Definición de Herramienta MCP

Crea una herramienta MCP que se conecte al servidor MCP de Microsoft Learn:
- **server_label**: Identificador para el servidor MCP
- **server_url**: Punto de acceso URL del servidor MCP
- **allowed_tools**: Lista opcional para restringir qué herramientas pueden ser utilizadas (una lista vacía permite todas las herramientas)

Esta herramienta permitirá al agente acceder a la documentación y recursos de Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Crear Agente y Ejecutar Conversación (Flujo sin Claves)

Esta sección completa demuestra el **flujo de trabajo de agente sin claves**:

1. **Crear Agente de IA**: Configura un agente con el modelo GPT-4.1 nano y herramientas MCP.
2. **Crear Hilo**: Establece un hilo de conversación para la comunicación.
3. **Enviar Mensaje**: Pregunta al agente sobre las diferencias entre Azure OpenAI y OpenAI.
4. **Gestionar Aprobaciones de Herramientas**: Aprueba automáticamente las llamadas a herramientas MCP cuando sea necesario.
5. **Supervisar la Ejecución**: Monitorea el progreso del agente y gestiona cualquier acción requerida.
6. **Mostrar Resultados**: Presenta los detalles de la conversación y el uso de herramientas.

**Características sin Claves:**
- ✅ **Sin secretos codificados** - Toda la autenticación se gestiona mediante la identidad de Azure.
- ✅ **Seguro por defecto** - Utiliza control de acceso basado en roles.
- ✅ **Despliegue simplificado** - No se requiere gestión de credenciales.
- ✅ **Amigable con auditorías** - Todo el acceso se rastrea a través de la identidad de Azure.

El agente usará herramientas MCP para acceder a recursos de Microsoft Learn con total seguridad y sin necesidad de gestionar claves API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
